<a href="https://colab.research.google.com/github/RuthBiney/BCI-Ampe_Game/blob/main/BCI_Ampe_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Step 1: Set Up Your Development Environment
###1. Install Python:
*  Download and install Python (version 3.8 or higher) from python.org.

###2. Install Required Libraries:
*   Open a terminal or command prompt and install the following libraries:




In [ ]:
pip install tensorflow keras numpy pandas scipy mne matplotlib seaborn plotly

##Step 2: Load and Preprocess the Dataset
###1. Download the Dataset:
*   Use the dataset I created myself whicch will be provided later

###2. Load the Dataset
*   Use Python to load the dataset


In [ ]:
import numpy as np
import mne

# Load EEG data using MNE-Python
raw = mne.io.read_raw_gdf('path_to_dataset.gdf', preload=True)

###3. Preprocess the Data:
*   Filter the data to remove noise and artifacts:

In [ ]:
raw.filter(7., 30., fir_design='firwin')  # Bandpass filter (7-30 Hz)

*   Normalize the data:



In [ ]:
from sklearn.preprocessing import StandardScaler

# Extract EEG data
eeg_data = raw.get_data().T  # Shape: (n_samples, n_channels)

# Normalize the data
scaler = StandardScaler()
eeg_data = scaler.fit_transform(eeg_data).T  # Shape: (n_channels, n_samples)

*   Segment the data into time windows:



In [ ]:
window_size = 128  # 1-second window at 128 Hz
segments = np.array([eeg_data[:, i:i+window_size] for i in range(0, eeg_data.shape[1] - window_size, window_size)])

# Print the shape of the segmented data
print(segments.shape)  # Should be (n_segments, n_channels, window_size)

*   Prepare Labels:

  Extract the labels for each segment (e.g., left hand, right hand, foot, tongue):

In [ ]:
# Example: Assuming labels are stored in a separate file
labels = np.load('path_to_labels.npy')  # Load labels
labels = labels[:segments.shape[0]]  # Ensure labels match the number of segments

###4. Save the Preprocessed Data
Save the preprocessed data and labels for later use:

In [ ]:
np.save('preprocessed_eeg_data.npy', segments)
np.save('labels.npy', labels)

##Step 3: Verify the Preprocessed Data
Before moving to model development, verify that the preprocessed data is correct:
###1. Check the shape of the segment data:

In [ ]:
print(segments.shape)  # Should be (n_segments, n_channels, window_size)

###2. Visualize a sample segment:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(segments[0].T)  # Plot the first segment
plt.title("Sample EEG Segment")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

##Step 4: Design the Hybrid CNN-RNN Model
###1. Import Libraries:
First, import the necessary libraries for building the model:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, Reshape

###2. Define the Model:
We’ll create a hybrid CNN-RNN model using TensorFlow/Keras. Here’s the step-by-step process:

###1. Input Layer:
*   Reshape the input data to match the expected format for the CNN (e.g., (n_channels, window_size, 1)).

###2. CNN Layers:
*   Use Conv2D layers to extract spatial features from the EEG signals.
*   Add MaxPooling2D layers to reduce the dimensionality of the features.

###3. RNN Layers:
*   Use LSTM layers to capture temporal dependencies in the EEG data.

###4. Output Layer:
*   Use a Dense layer with a softmax activation function to classify the motor imagery tasks.



In [ ]:
# Define the model
model = Sequential()

# Reshape input for CNN (n_channels, window_size, 1)
model.add(Reshape((segments.shape[1], segments.shape[2], 1), input_shape=(segments.shape[1], segments.shape[2])))

# CNN Layers
model.add(Conv2D(32, (3, 3), activation='relu'))  # 32 filters, 3x3 kernel
model.add(MaxPooling2D((2, 2)))  # Max pooling with 2x2 window
model.add(Flatten())  # Flatten the output for RNN input

# Reshape for RNN (n_timesteps, n_features)
model.add(Reshape((-1, 32)))  # Reshape to (n_timesteps, n_features)

# RNN Layers
model.add(LSTM(64, return_sequences=True))  # 64 LSTM units, return sequences
model.add(LSTM(32))  # 32 LSTM units

# Output Layer
model.add(Dense(4, activation='softmax'))  # 4 classes for motor imagery tasks

###3. Compile the Model:
Compile the model by specifying the optimizer, loss function, and evaluation metric:

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

###4. Print the Model Summary
Print a summary of the model to verify its architecture:

In [ ]:
model.summary()

##Step 5: Prepare the Data for Training
Before training the model, ensure the data is in the correct format:

###1. Convert Labels to Categorical:
*   Use one-hot encoding for the labels:

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

###2. Split the Data:
*   Split the preprocessed data into training and testing sets (if not already done):


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(segments, labels, test_size=0.2, random_state=42)

##Step 6: Train the Model
Train the model using the preprocessed data:

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

##Step 7: Evaluate the Model
Evaluate the model’s performance on the test set:

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

##Step 8: Visualize Training Results
Plot the training and validation accuracy/loss to analyze the model’s performance:

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()